# Merging the Marine Magnetics Database

## Imports

In [1]:
import os
import csv
from typing import List
from pathlib import Path
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Useful Functions

In [3]:
EXPECTED_COLS = [
    "Reading_Date","Reading_Time","Magnetic_Field","Signal_Quality","Depth","Leak",
    "Measurement_Time","Signal_Strength","Gradient_Condition","Weak_Signal","Poor_Reading",
    "Instrument_Mistuned","Reading_ID","System_Time","Line_Name","Error",
    "Mag_Latitude","Mag_Longitude","Mag_Easting","Mag_Northing","Mag_Zone",
    "GPS_Latitude","GPS_Longitude","GPS_Easting","GPS_Northing","GPS_Zone",
    "Marker","Marker_Time"
]
N_COLS = len(EXPECTED_COLS)
NA_TOKENS = {"", " ", "*"}

def _strict_read_rows(path: Path):
    """Read CSV by hand:
       - drop lines starting with '/'
       - first non-'/' line is header (ignored; we enforce EXPECTED_COLS)
       - ensure each data row has exactly N_COLS fields (pad/truncate)
    """
    rows = []
    with open(path, "r", encoding="utf-8-sig", newline="") as f:
        rdr = csv.reader(f, delimiter=",", quotechar='"', skipinitialspace=True)
        # skip metadata lines beginning with '/'
        data_started = False
        for raw in rdr:
            if not raw:
                continue
            if raw[0].startswith("/"):
                continue
            if not data_started:
                # first non-comment line is the header in file; ignore and enforce ours
                data_started = True
                continue
            # normalize row length
            if len(raw) < N_COLS:
                raw = raw + [""] * (N_COLS - len(raw))
            elif len(raw) > N_COLS:
                raw = raw[:N_COLS]
            rows.append(raw)
    return rows

def _to_dataframe(rows):
    df = pd.DataFrame(rows, columns=EXPECTED_COLS)

    # Strip whitespace from all string values
    obj_cols = df.select_dtypes(include="object").columns
    for c in obj_cols:
        df[c] = df[c].str.strip()

    # Replace NA tokens with pandas NA
    df.replace(list(NA_TOKENS), pd.NA, inplace=True)

    # Coerce numerics (zones stay as strings)
    numeric_cols = [
        "Magnetic_Field","Signal_Quality","Depth","Leak","Measurement_Time","Signal_Strength",
        "Mag_Latitude","Mag_Longitude","Mag_Easting","Mag_Northing",
        "GPS_Latitude","GPS_Longitude","GPS_Easting","GPS_Northing","Reading_ID"
    ]
    for c in numeric_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Validate that Mag_Latitude is sane (detects any lingering shift)
    bad_lat = df["Mag_Latitude"].dropna().abs().gt(90)
    if bad_lat.any():
        raise ValueError("Column misalignment persists: Mag_Latitude out of range. Check source file.")

    return df

def _build_rel_timestamp(df: pd.DataFrame) -> None:
    """Create timestamp_s with t0 at first valid time in file."""
    abs_time = None
    if {"Reading_Date","Reading_Time"}.issubset(df.columns):
        abs_time = pd.to_datetime(
            df["Reading_Date"].astype(str) + " " + df["Reading_Time"].astype(str),
            dayfirst=True, errors="coerce"
        )
    elif "System_Time" in df.columns:
        abs_time = pd.to_datetime(df["System_Time"], dayfirst=True, errors="coerce")

    if abs_time is not None and not abs_time.isna().all():
        t0 = abs_time.min()
        df["timestamp_s"] = (abs_time - t0).dt.total_seconds().astype("float")
    else:
        df["timestamp_s"] = pd.NA

def read_mmc_csv_strict(path: Path) -> pd.DataFrame:
    """Robust import that prevents the Error→Mag_Latitude shift."""
    rows = _strict_read_rows(path)
    df = _to_dataframe(rows)
    _build_rel_timestamp(df)
    df["source_file"] = path.name
    return df

def read_all_mmc_csvs(data_dir: str) -> pd.DataFrame:
    files = sorted(Path(data_dir).glob("*.csv"))
    frames = []
    for p in files:
        try:
            df = read_mmc_csv_strict(p)
            frames.append(df)
            print(f"Loaded: {p.name}")
        except Exception as e:
            print(f"Failed: {p.name} -> {e}")

    if not frames:
        return pd.DataFrame()

    df_all = pd.concat(frames, ignore_index=True)

    # Assign Line XX per source_file (stable order by sorted files)
    mapping = {fname: f"Line {str(i+1).zfill(2)}"
               for i, fname in enumerate(df_all["source_file"].unique())}
    df_all["Line_Name"] = df_all["source_file"].map(mapping)
    return df_all


def fix_shifted_headers(df):

    """
    Shift column NAMES left starting at 'Error' through 'Marker_Time',
    so that values currently under those columns get the correct labels.
    Then drop the last 'Marker_Time' (duplicate introduced by the shift).
    """
    cols = list(df.columns)

    # sanity checks
    if "Error" not in cols or "Marker_Time" not in cols:
        raise ValueError("Expected columns 'Error' and 'Marker_Time' not found.")
    start = cols.index("Error")
    end   = cols.index("Marker_Time")  # inclusive

    if end <= start:
        raise ValueError("Unexpected column order: 'Marker_Time' occurs before 'Error'.")

    # Shift the header names left by 1 within [start, end]
    # Error -> Mag_Latitude, Mag_Latitude -> Mag_Longitude, ..., Marker -> Marker_Time
    # Marker_Time stays Marker_Time (so we end up with two 'Marker_Time' headers)
    for i in range(start, end):
        cols[i] = cols[i + 1]
    cols[end] = "Marker_Time"

    # Apply the new header names
    df.columns = cols

    # Drop ONLY the last 'Marker_Time' (the one originally at position 'end')
    keep_idx = [i for i in range(len(cols)) if i != end]
    df = df.iloc[:, keep_idx].copy()

    return df


## Importing the data

In [4]:
## Change here for your local data directory
data_dir = '/content/drive/MyDrive/labgeo-marco/data/Magnetometry'


In [5]:
df_all = read_all_mmc_csvs(data_dir)
df_all = fix_shifted_headers(df_all)
# print(df_all.head())

Loaded: CAN_250820_S6_C.csv
Loaded: Can_250820_D2_C.csv
Loaded: Can_250820_D3_c.csv
Loaded: Can_250820_S8_c.csv
Loaded: Can_250820_S9_c.csv
Loaded: can250819c1a.csv
Loaded: can250819d1a.csv
Loaded: can250819d2a.csv
Loaded: can250819s1a.csv
Loaded: can250819s2a.csv
Loaded: can250819s3a.csv
Loaded: can250819s4a.csv
Loaded: can250819s5a.csv
Loaded: can250821_D2_b.csv
Loaded: can_250820_C1_c.csv
Loaded: can_250820_C2_c.csv
Loaded: can_250820_D1_c.csv
Loaded: can_250820_D4_c_E.csv
Loaded: can_250820_D5_c.csv
Loaded: can_250820_DG_c.csv
Loaded: can_250820_S10_c.csv
Loaded: can_250820_S11_c.csv
Loaded: can_250820_S1_c.csv
Loaded: can_250820_S2_c.csv
Loaded: can_250820_S3_c.csv
Loaded: can_250820_S5_c.csv
Loaded: can_250820_S7_c.csv
Loaded: can_250820_d4_c.csv
Loaded: can_250821_C2.csv
Loaded: can_250821_D1_b_2.csv
Loaded: can_250821_D3_B.csv
Loaded: can_250821_D4_B.csv
Loaded: can_250821_S1_B..csv
Loaded: can_250821_S2_B.csv
Loaded: can_250821_S3_B.csv
Loaded: can_250821_S4_B.csv
Loaded: can_

In [6]:
df_all

,Reading_Date,Reading_Time,Magnetic_Field,Signal_Quality,Depth,Leak,Measurement_Time,Signal_Strength,Gradient_Condition,Weak_Signal,Poor_Reading,Instrument_Mistuned,Reading_ID,System_Time,Line_Name,Mag_Latitude,Mag_Longitude,Mag_Easting,Mag_Northing,Mag_Zone,GPS_Latitude,GPS_Longitude,GPS_Easting,GPS_Northing,GPS_Zone,Marker,Marker_Time,timestamp_s,source_file
0,20-Aug-2025,17:36:36.700,22759.127,79,0.0,0,215,119,NaN,NaN,NaN,NaN,2,20-Aug-2025 17:36:38.207,Line 01,-25.04215631,-48.017993,800898.50,7227067.90,NaN,-25.04222737,-48.018367,800860.58,7227060.86,NaN,NaN,NaN,0.0,CAN_250820_S6_C.csv
1,20-Aug-2025,17:36:37.200,22757.742,79,0.0,0,215,122,NaN,NaN,NaN,NaN,3,20-Aug-2025 17:36:38.660,Line 01,-25.04215727,-48.017996,800898.20,7227067.80,NaN,-25.04222760,-48.018370,800860.27,7227060.84,NaN,NaN,NaN,0.5,CAN_250820_S6_C.csv
2,20-Aug-2025,17:36:37.700,22756.970,89,0.0,0,215,129,NaN,NaN,NaN,NaN,4,20-Aug-2025 17:36:39.157,Line 01,-25.04215968,-48.018003,800897.46,7227067.55,NaN,-25.04222816,-48.018378,800859.48,7227060.79,NaN,NaN,NaN,1.0,CAN_250820_S6_C.csv
3,20-Aug-2025,17:36:38.700,22753.489,89,0.0,0,215,133,NaN,NaN,NaN,NaN,5,20-Aug-2025 17:36:40.177,Line 01,-25.04216432,-48.018018,800895.97,7227067.07,NaN,-25.04222907,-48.018393,800857.90,7227060.73,NaN,NaN,NaN,2.0,CAN_250820_S6_C.csv
4,20-Aug-2025,17:36:39.200,22750.916,89,0.0,0,215,134,NaN,NaN,NaN,NaN,6,20-Aug-2025 17:36:40.687,Line 01,-25.04216656,-48.018025,800895.22,7227066.83,NaN,-25.04222942,-48.018401,800857.11,7227060.70,NaN,NaN,NaN,2.5,CAN_250820_S6_C.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79341,21-Aug-2025,18:00:45.200,22585.829,89,0.0,0,215,129,NaN,NaN,NaN,NaN,26,21-Aug-2025 18:00:45.510,Line 38,-24.94326828,-47.911882,205945.90,7238181.05,NaN,-24.94306156,-47.911574,205976.53,7238204.63,NaN,NaN,NaN,15.0,can_250821_S6_B..csv
79342,21-Aug-2025,18:00:45.700,22585.503,89,0.0,0,215,133,NaN,NaN,NaN,NaN,27,21-Aug-2025 18:00:46.023,Line 38,-24.94326420,-47.911875,205946.62,7238181.52,NaN,-24.94305649,-47.911568,205977.17,7238205.21,NaN,NaN,NaN,15.5,can_250821_S6_B..csv
79343,21-Aug-2025,18:00:46.200,22585.352,89,0.0,0,215,133,NaN,NaN,NaN,NaN,28,21-Aug-2025 18:00:46.543,Line 38,-24.94326015,-47.911868,205947.33,7238181.99,NaN,-24.94305158,-47.911561,205977.82,7238205.76,NaN,NaN,NaN,16.0,can_250821_S6_B..csv
79344,21-Aug-2025,18:00:46.700,22584.775,89,0.0,0,215,133,NaN,NaN,NaN,NaN,29,21-Aug-2025 18:00:47.050,Line 38,-24.94325616,-47.911861,205948.03,7238182.44,NaN,-24.94304691,-47.911555,205978.46,7238206.30,NaN,NaN,NaN,16.5,can_250821_S6_B..csv


In [7]:
df_all.shape

(79346, 29)

In [8]:
df_all.isna().sum()

,0
Reading_Date,0
Reading_Time,0
Magnetic_Field,0
Signal_Quality,0
Depth,0
Leak,0
Measurement_Time,0
Signal_Strength,0
Gradient_Condition,78303
Weak_Signal,79346


In [9]:
df_all['source_file'].unique().shape

(38,)

In [10]:
df_all.columns

Index(['Reading_Date', 'Reading_Time', 'Magnetic_Field', 'Signal_Quality',
       'Depth', 'Leak', 'Measurement_Time', 'Signal_Strength',
       'Gradient_Condition', 'Weak_Signal', 'Poor_Reading',
       'Instrument_Mistuned', 'Reading_ID', 'System_Time', 'Line_Name',
       'Mag_Latitude', 'Mag_Longitude', 'Mag_Easting', 'Mag_Northing',
       'Mag_Zone', 'GPS_Latitude', 'GPS_Longitude', 'GPS_Easting',
       'GPS_Northing', 'GPS_Zone', 'Marker', 'Marker_Time', 'timestamp_s',
       'source_file'],
      dtype='object')

### Survey`s period

In [11]:
df_all['Reading_Date'].unique()

array(['20-Aug-2025', '19-Aug-2025', '21-Aug-2025', '29-Nov-2025'],
      dtype=object)

In [12]:
df_all[df_all['Reading_Date'] == '29-Nov-2025']

,Reading_Date,Reading_Time,Magnetic_Field,Signal_Quality,Depth,Leak,Measurement_Time,Signal_Strength,Gradient_Condition,Weak_Signal,Poor_Reading,Instrument_Mistuned,Reading_ID,System_Time,Line_Name,Mag_Latitude,Mag_Longitude,Mag_Easting,Mag_Northing,Mag_Zone,GPS_Latitude,GPS_Longitude,GPS_Easting,GPS_Northing,GPS_Zone,Marker,Marker_Time,timestamp_s,source_file
73140,29-Nov-2025,17:43:43.200,22694.236,89,0.0,0,215,133,<NA>,<NA>,<NA>,<NA>,6694,21-Aug-2025 17:43:43.490,Line 31,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,8644905.0,can_250821_D3_B.csv
73141,29-Nov-2025,17:43:43.700,22694.723,89,0.0,0,215,134,<NA>,<NA>,<NA>,<NA>,6695,21-Aug-2025 17:43:43.963,Line 31,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,8644905.5,can_250821_D3_B.csv
73142,29-Nov-2025,17:43:45.200,22694.664,89,0.0,0,215,132,<NA>,<NA>,<NA>,<NA>,6696,21-Aug-2025 17:43:45.463,Line 31,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,8644907.0,can_250821_D3_B.csv
73143,29-Nov-2025,17:43:45.700,22695.248,89,0.0,0,215,130,<NA>,<NA>,<NA>,<NA>,6697,21-Aug-2025 17:43:45.960,Line 31,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,8644907.5,can_250821_D3_B.csv
73144,29-Nov-2025,17:43:46.200,22694.667,89,0.0,0,215,128,<NA>,<NA>,<NA>,<NA>,6698,21-Aug-2025 17:43:46.480,Line 31,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,8644908.0,can_250821_D3_B.csv
73145,29-Nov-2025,17:43:46.700,22694.444,79,0.0,0,215,127,<NA>,<NA>,<NA>,<NA>,6699,21-Aug-2025 17:43:47.010,Line 31,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,8644908.5,can_250821_D3_B.csv
73146,29-Nov-2025,17:43:47.200,22695.969,89,0.0,0,215,129,<NA>,<NA>,<NA>,<NA>,6700,21-Aug-2025 17:43:47.510,Line 31,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,8644909.0,can_250821_D3_B.csv
73147,29-Nov-2025,17:43:47.700,22695.631,89,0.0,0,215,130,<NA>,<NA>,<NA>,<NA>,6701,21-Aug-2025 17:43:47.960,Line 31,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,8644909.5,can_250821_D3_B.csv
73148,29-Nov-2025,17:43:48.200,22696.350,79,0.0,0,215,127,<NA>,<NA>,<NA>,<NA>,6702,21-Aug-2025 17:43:48.460,Line 31,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,8644910.0,can_250821_D3_B.csv
73149,29-Nov-2025,17:43:56.700,22695.809,69,0.0,0,215,101,<NA>,<NA>,<NA>,<NA>,6703,21-Aug-2025 17:43:56.983,Line 31,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,8644918.5,can_250821_D3_B.csv


In [13]:
df_all.loc[df_all['Reading_Date'] == '29-Nov-2025', 'Reading_Date'] = '21-Aug-2025'

In [14]:
print(f'Survey started at : {df_all['Reading_Date'].min()} - {df_all['Reading_Time'].min()}')
print(f'Survey ended at : {df_all['Reading_Date'].max()} - {df_all['Reading_Time'].max()}')

Survey started at : 19-Aug-2025 - 11:16:35.200
Survey ended at : 21-Aug-2025 - 20:15:04.700


## Merging the Geomagnetic Data

Now, we will merge the geomagnetic data from the nearest observatory to our database. We are going to use the Vassouras geomagnetic observatory (VSS) in Rio the Janeiro, Brazil.


Data can be downloaded folling the steps presented in the [INTERMAGNET website](https://imag-data.bgs.ac.uk/GIN_V1/GINForms2?observatoryIagaCode=MID&publicationState=Best+available&dataStartDate=2025-10-09&dataDuration=1&samplesPerDay=minute&submitValue=Bulk+Download&request=DataView).

In [15]:
import re
from pathlib import Path
import pandas as pd
import numpy as np

def load_intermagnet_files(paths, tz="UTC"):
    """
    Load one or many IAGA-2002 style files (e.g., VSS daily files) into a single DataFrame.

    Parameters
    ----------
    paths : str | Path | list[str|Path]
        File path(s) or glob pattern(s). Examples:
          - "data/VSS_20250821.min"
          - "data/VSS_2025*.min"
          - ["day1.min", "day2.min"]
    tz : str or None
        Timezone for the returned DatetimeIndex. Default "UTC".
        If None, the index will be naive (no timezone).

    Returns
    -------
    df_all : pd.DataFrame
        Concatenated DataFrame from all files.
        Index: DatetimeIndex named 'datetime' (tz-aware if tz provided).
        Columns: as provided in the files (DATE, TIME, DOY, VSSX, VSSY, VSSZ, VSSF, ...).
    """
    # Normalize input to a list of string patterns
    if isinstance(paths, (str, Path)):
        patterns = [str(paths)]
    else:
        patterns = [str(p) for p in paths]

    # Expand globs
    files = []
    for pat in patterns:
        if any(ch in pat for ch in "*?[]"):
            files.extend([str(p) for p in Path().glob(pat)])
        else:
            files.append(pat)

    files = sorted(set(files))
    if not files:
        raise FileNotFoundError("No IAGA-2002 files found for the given path(s).")

    dfs = []
    for fp in files:
        with open(fp, "r", encoding="utf-8", errors="ignore") as f:
            lines = f.readlines()

        # Find header row (starts with "DATE  TIME")
        header_idx = None
        for i, line in enumerate(lines):
            if re.match(r"^\s*DATE\s+TIME\b", line):
                header_idx = i
                break
        if header_idx is None:
            raise ValueError(f"Could not find 'DATE  TIME' header in: {fp}")

        # Read table from header line onward.
        # Replace the visual '|' separators to avoid them being read as literal tokens.
        block = "".join(lines[header_idx:]).replace("|", " ")

        df = pd.read_csv(
            pd.io.common.StringIO(block),
            sep=r"\s+",
            engine="python"
        )

        # Build datetime from DATE + TIME
        dt = pd.to_datetime(
            df["DATE"].astype(str) + " " + df["TIME"].astype(str),
            errors="coerce", utc=True
        )
        if dt.isna().all():
            # Fallback: some files may use comma as decimal separator in TIME seconds
            dt = pd.to_datetime(
                (df["DATE"].astype(str) + " " + df["TIME"].astype(str)).str.replace(",", "."),
                errors="coerce", utc=True
            )

        df = df.assign(datetime=dt).dropna(subset=["datetime"]).set_index("datetime")
        df.index.name = "datetime"
        df = df.sort_index()

        # Apply requested timezone
        if tz is None:
            # make index naive (remove tz info)
            df.index = df.index.tz_localize(None)
        else:
            # ensure UTC first, then convert if needed
            if df.index.tz is None:
                df.index = df.index.tz_localize("UTC")
            if tz.upper() != "UTC":
                df = df.tz_convert(tz)
            else:
                # keep UTC
                pass

        dfs.append(df)

    # Concatenate all days
    df_all = pd.concat(dfs, axis=0)
    # Drop exact duplicate timestamps (keep first)
    df_all = df_all[~df_all.index.duplicated(keep="first")].sort_index()

    return df_all


In [16]:
 geomag_data_path = '/content/drive/MyDrive/labgeo-marco/data/VSS'

In [17]:
geomag_files = [f for f in os.listdir(geomag_data_path) if os.path.isfile(os.path.join(geomag_data_path, f))]
geomag_files_path = []
for f in geomag_files:
    geomag_files_path.append(os.path.join(geomag_data_path, f))

geomag_files_path

['/content/drive/MyDrive/labgeo-marco/data/VSS/vss20250821qmin.min',
 '/content/drive/MyDrive/labgeo-marco/data/VSS/vss20250820qmin.min',
 '/content/drive/MyDrive/labgeo-marco/data/VSS/vss20250819qmin.min']

In [18]:
df_geomag = load_intermagnet_files(geomag_files_path)
df_geomag

,DATE,TIME,DOY,VSSX,VSSY,VSSZ,VSSF
datetime,,,,,,,
2025-08-19 00:00:00+00:00,2025-08-19,00:00:00.000,231,15879.37,-6741.65,-15516.16,99999.00
2025-08-19 00:01:00+00:00,2025-08-19,00:01:00.000,231,15878.89,-6741.34,-15516.23,99999.00
2025-08-19 00:02:00+00:00,2025-08-19,00:02:00.000,231,15878.75,-6741.57,-15516.32,99999.00
2025-08-19 00:03:00+00:00,2025-08-19,00:03:00.000,231,15878.62,-6741.56,-15516.53,99999.00
2025-08-19 00:04:00+00:00,2025-08-19,00:04:00.000,231,15877.95,-6741.60,-15516.66,99999.00
...,...,...,...,...,...,...,...
2025-08-21 23:55:00+00:00,2025-08-21,23:55:00.000,233,15878.49,-6744.35,-15515.44,23202.34
2025-08-21 23:56:00+00:00,2025-08-21,23:56:00.000,233,15878.23,-6744.37,-15515.57,23202.22
2025-08-21 23:57:00+00:00,2025-08-21,23:57:00.000,233,15877.78,-6744.47,-15515.80,23202.09


### Visualizing the geomagnetic data

In [19]:
# ==== Interactive multi-plot browser for observatory components (separate subplots) ====
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from ipywidgets import widgets, HBox, VBox
from IPython.display import display

def make_geomag_time_browser_multi(
    obs_df,
    components=("VSSX","VSSY","VSSZ","VSSF"),
    sentinels=(99999.0, 88888.0),
    title="Geomagnetic components (UTC)",
):
    """
    Interactive viewer: one subplot per component, shared time axis.

    Controls:
      - Day dropdown (UTC date from index)
      - Time window slider (minutes within the selected day)

    Parameters
    ----------
    obs_df : pd.DataFrame
        Observatory data with a DatetimeIndex (tz-aware recommended, UTC preferred),
        columns like 'VSSX','VSSY','VSSZ','VSSF'.
    components : tuple[str]
        Columns to plot. Non-existent columns are ignored.
    sentinels : tuple[float]
        Values to treat as NaN (e.g., 99999.0, 88888.0).
    title : str
        Figure title prefix.
    """
    if not isinstance(obs_df.index, pd.DatetimeIndex):
        raise ValueError("obs_df must have a DatetimeIndex.")

    # Ensure UTC tz-awareness for clean slicing
    if obs_df.index.tz is None:
        obs = obs_df.copy()
        obs.index = obs.index.tz_localize("UTC")
    else:
        obs = obs_df.tz_convert("UTC").copy()

    # Keep only present components
    comps = [c for c in components if c in obs.columns]
    if not comps:
        raise ValueError("None of the requested components are present in obs_df.")

    # Numeric + mask sentinels
    for c in comps:
        obs[c] = pd.to_numeric(obs[c], errors="coerce")
        if sentinels:
            obs[c] = obs[c].replace({s: np.nan for s in sentinels})

    # Build day list (UTC)
    obs["_date_"] = obs.index.normalize().date
    days = sorted(pd.unique(obs["_date_"]))
    if not days:
        raise ValueError("No valid timestamps found.")

    # ---- widgets ----
    day_dd = widgets.Dropdown(
        options=days, value=days[0],
        description="Day (UTC):", layout=widgets.Layout(width="35%")
    )
    win_slider = widgets.IntRangeSlider(
        value=(0, 1440), min=0, max=1440, step=1,
        description="Window (min):", readout=False, layout=widgets.Layout(width="50%")
    )
    win_label = widgets.HTML()
    out = widgets.Output()

    def _fmt_minutes(m):
        h = m // 60
        mm = m % 60
        return f"{int(h):02d}:{int(mm):02d}"

    def redraw(*_):
        with out:
            out.clear_output(wait=True)

            day = pd.to_datetime(str(day_dd.value))
            m0, m1 = win_slider.value
            win_label.value = f"<b>Window:</b> {_fmt_minutes(m0)} – {_fmt_minutes(m1)} (UTC)"

            start = pd.Timestamp(day).tz_localize("UTC") + pd.Timedelta(minutes=m0)
            end   = pd.Timestamp(day).tz_localize("UTC") + pd.Timedelta(minutes=m1)

            d = obs.loc[(obs.index >= start) & (obs.index <= end), comps]
            if d.empty:
                fig, ax = plt.subplots(figsize=(10, 3), dpi=130)
                ax.text(0.5, 0.5, "No data in selected window.", ha="center", va="center", transform=ax.transAxes)
                ax.set_axis_off()
                plt.show()
                return

            # Figure height scales with number of components
            fig_h = 2.6 * len(comps) + 1.0
            fig, axes = plt.subplots(
                nrows=len(comps), ncols=1, figsize=(11, fig_h), dpi=130,
                sharex=True, constrained_layout=True
            )
            if len(comps) == 1:
                axes = [axes]

            for ax, c in zip(axes, comps):
                ax.plot(d.index, d[c], lw=1.1, label=c)
                ax.set_ylabel("nT")
                ax.grid(True, ls=":")
                ax.legend(loc="best", fontsize=9)

                # Avoid scientific notation
                ax.yaxis.set_major_formatter(mticker.ScalarFormatter(useMathText=False))
                ax.ticklabel_format(style="plain", axis="y")

            axes[-1].set_xlabel("Time (UTC)")
            fig.suptitle(f"{title} — {day_dd.value} {_fmt_minutes(m0)}–{_fmt_minutes(m1)}", fontsize=12)
            plt.show()

    # Wire up
    day_dd.observe(redraw, names="value")
    win_slider.observe(redraw, names="value")

    # Initial draw
    redraw()

    # Layout
    controls = HBox([day_dd, win_slider, win_label])
    display(VBox([controls, out]))

In [20]:
make_geomag_time_browser_multi(
    df_geomag,                         # your DataFrame (DatetimeIndex, UTC preferred)
    components=("VSSX","VSSY","VSSZ","VSSF")  # pick any subset/order you want
)

### Merging geomag data to the survey database

In [21]:
import pandas as pd
import numpy as np

def _to_naive_utc(x):
    """Return a pandas DatetimeIndex/Series converted to UTC then made tz-naive."""
    if isinstance(x, pd.Series):
        if x.dt.tz is None:
            return x
        return x.dt.tz_convert("UTC").dt.tz_localize(None)
    # DatetimeIndex
    if x.tz is None:
        return x
    return x.tz_convert("UTC").tz_localize(None)

def merge_intermagnet_to_survey(
    survey_df,
    obs_df,
    date_col="Reading_Date",
    time_col="Reading_Time",
    date_fmt="%d-%b-%Y %H:%M:%S.%f",
    tz_survey="UTC",
    suffix="_vss",
    interp_method="time",        # 'time' recommended
    nearest_tolerance="5min",    # fallback tolerance
    allow_edge_fill=True         # ffill/bfill at edges if just outside coverage
):
    """
    Merge observatory data onto survey samples by:
      1) time interpolation at survey timestamps
      2) nearest-neighbor fallback within `nearest_tolerance`
      3) optional edge fill if just outside obs coverage
    Keeps original obs_df column names (appends `suffix`). Adds `__survey_datetime_utc`.
    """

    # ---- 1) Parse survey datetimes (-> UTC tz-aware) ----
    if date_fmt:
        survey_dt = pd.to_datetime(
            survey_df[date_col].astype(str) + " " + survey_df[time_col].astype(str),
            format=date_fmt, errors="coerce"
        )
    else:
        survey_dt = pd.to_datetime(
            survey_df[date_col].astype(str) + " " + survey_df[time_col].astype(str),
            errors="coerce", infer_datetime_format=True
        )
    if survey_dt.isna().any():
        bad = survey_df.loc[survey_dt.isna(), [date_col, time_col]].head(5)
        raise ValueError("Failed to parse some survey timestamps:\n" + bad.to_string(index=False))

    # localize/convert
    if survey_dt.dt.tz is None:
        survey_dt = survey_dt.dt.tz_localize(tz_survey or "UTC")
    else:
        survey_dt = survey_dt.dt.tz_convert(tz_survey or "UTC")
    survey_dt = survey_dt.dt.tz_convert("UTC")

    # Use a DatetimeIndex for alignment (avoids tz loss)
    survey_ix = pd.DatetimeIndex(survey_dt)

    # ---- 2) Prepare observatory DF (UTC tz-aware, numeric) ----
    obs = obs_df.copy()
    if obs.index.tz is None:
        obs.index = obs.index.tz_localize("UTC")
    else:
        obs = obs.tz_convert("UTC")
    obs = obs[~obs.index.duplicated(keep="first")].sort_index()

    for c in obs.columns:
        obs[c] = pd.to_numeric(obs[c], errors="coerce").replace(
            {88888: np.nan, 99999: np.nan, 88888.0: np.nan, 99999.0: np.nan}
        )

    # No overlap? return with empty obs cols (explicit NaNs)
    if (survey_ix.max() < obs.index.min()) or (survey_ix.min() > obs.index.max()):
        out = survey_df.copy()
        out["__survey_datetime_utc"] = survey_ix
        for c in obs.columns:
            out[f"{c}{suffix}"] = np.nan
        return out

    # ---- 3) Interpolate exactly at survey timestamps ----
    obs_interp = (
        obs.reindex(obs.index.union(survey_ix))   # include survey times
           .sort_index()
           .interpolate(method=interp_method, limit_direction="both")
           .reindex(survey_ix)                    # keep only survey times (tz-aware)
    )

    # ---- 4) Nearest-neighbor fallback for any remaining NaNs ----
    nan_mask = obs_interp.isna().all(axis=1)
    if nan_mask.any():
        missing_idx = obs_interp.index[nan_mask]  # tz-aware UTC

        # Build frames for merge_asof on UTC-naive duplicates
        s = pd.DataFrame({"__dt": _to_naive_utc(missing_idx)})
        o = obs.reset_index().rename(columns={"index": "__dt"})
        o["__dt"] = _to_naive_utc(o["__dt"])

        near = pd.merge_asof(
            s.sort_values("__dt"),
            o.sort_values("__dt"),
            on="__dt",
            direction="nearest",
            tolerance=pd.Timedelta(nearest_tolerance)
        ).set_index("__dt")

        # Map back to original tz-aware missing index
        near.index = near.index.tz_localize("UTC")
        obs_interp.loc[nan_mask, :] = near.reindex(missing_idx).values

    # ---- 5) Optional edge fill (tiny overhangs) ----
    if allow_edge_fill:
        obs_interp = obs_interp.ffill().bfill()

    # ---- 6) Attach to survey ----
    obs_interp = obs_interp.rename(columns={c: f"{c}{suffix}" for c in obs_interp.columns})

    out = survey_df.copy()
    out["__survey_datetime_utc"] = survey_ix
    out = pd.concat([out.reset_index(drop=True),
                     obs_interp.reset_index(drop=True)], axis=1)
    return out


In [22]:
df_all_complete = merge_intermagnet_to_survey(df_all, df_geomag, date_col='Reading_Date', time_col='Reading_Time')

df_all_complete

,Reading_Date,Reading_Time,Magnetic_Field,Signal_Quality,Depth,Leak,Measurement_Time,Signal_Strength,Gradient_Condition,Weak_Signal,Poor_Reading,Instrument_Mistuned,Reading_ID,System_Time,Line_Name,Mag_Latitude,Mag_Longitude,Mag_Easting,Mag_Northing,Mag_Zone,GPS_Latitude,GPS_Longitude,GPS_Easting,GPS_Northing,GPS_Zone,Marker,Marker_Time,timestamp_s,source_file,__survey_datetime_utc,DATE_vss,TIME_vss,DOY_vss,VSSX_vss,VSSY_vss,VSSZ_vss,VSSF_vss
0,20-Aug-2025,17:36:36.700,22759.127,79,0.0,0,215,119,NaN,NaN,NaN,NaN,2,20-Aug-2025 17:36:38.207,Line 01,-25.04215631,-48.017993,800898.50,7227067.90,NaN,-25.04222737,-48.018367,800860.58,7227060.86,NaN,NaN,NaN,0.0,CAN_250820_S6_C.csv,2025-08-20 17:36:36.700000+00:00,NaN,NaN,232.0,15878.953200,-6747.600383,-15509.501750,23199.548250
1,20-Aug-2025,17:36:37.200,22757.742,79,0.0,0,215,122,NaN,NaN,NaN,NaN,3,20-Aug-2025 17:36:38.660,Line 01,-25.04215727,-48.017996,800898.20,7227067.80,NaN,-25.04222760,-48.018370,800860.27,7227060.84,NaN,NaN,NaN,0.5,CAN_250820_S6_C.csv,2025-08-20 17:36:37.200000+00:00,NaN,NaN,232.0,15878.951200,-6747.597800,-15509.503000,23199.547000
2,20-Aug-2025,17:36:37.700,22756.970,89,0.0,0,215,129,NaN,NaN,NaN,NaN,4,20-Aug-2025 17:36:39.157,Line 01,-25.04215968,-48.018003,800897.46,7227067.55,NaN,-25.04222816,-48.018378,800859.48,7227060.79,NaN,NaN,NaN,1.0,CAN_250820_S6_C.csv,2025-08-20 17:36:37.700000+00:00,NaN,NaN,232.0,15878.949200,-6747.595217,-15509.504250,23199.545750
3,20-Aug-2025,17:36:38.700,22753.489,89,0.0,0,215,133,NaN,NaN,NaN,NaN,5,20-Aug-2025 17:36:40.177,Line 01,-25.04216432,-48.018018,800895.97,7227067.07,NaN,-25.04222907,-48.018393,800857.90,7227060.73,NaN,NaN,NaN,2.0,CAN_250820_S6_C.csv,2025-08-20 17:36:38.700000+00:00,NaN,NaN,232.0,15878.945200,-6747.590050,-15509.506750,23199.543250
4,20-Aug-2025,17:36:39.200,22750.916,89,0.0,0,215,134,NaN,NaN,NaN,NaN,6,20-Aug-2025 17:36:40.687,Line 01,-25.04216656,-48.018025,800895.22,7227066.83,NaN,-25.04222942,-48.018401,800857.11,7227060.70,NaN,NaN,NaN,2.5,CAN_250820_S6_C.csv,2025-08-20 17:36:39.200000+00:00,NaN,NaN,232.0,15878.943200,-6747.587467,-15509.508000,23199.542000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79341,21-Aug-2025,18:00:45.200,22585.829,89,0.0,0,215,129,NaN,NaN,NaN,NaN,26,21-Aug-2025 18:00:45.510,Line 38,-24.94326828,-47.911882,205945.90,7238181.05,NaN,-24.94306156,-47.911574,205976.53,7238204.63,NaN,NaN,NaN,15.0,can_250821_S6_B..csv,2025-08-21 18:00:45.200000+00:00,NaN,NaN,233.0,15877.476133,-6752.511800,-15522.040933,23208.357733
79342,21-Aug-2025,18:00:45.700,22585.503,89,0.0,0,215,133,NaN,NaN,NaN,NaN,27,21-Aug-2025 18:00:46.023,Line 38,-24.94326420,-47.911875,205946.62,7238181.52,NaN,-24.94305649,-47.911568,205977.17,7238205.21,NaN,NaN,NaN,15.5,can_250821_S6_B..csv,2025-08-21 18:00:45.700000+00:00,NaN,NaN,233.0,15877.466467,-6752.510050,-15522.043267,23208.352067
79343,21-Aug-2025,18:00:46.200,22585.352,89,0.0,0,215,133,NaN,NaN,NaN,NaN,28,21-Aug-2025 18:00:46.543,Line 38,-24.94326015,-47.911868,205947.33,7238181.99,NaN,-24.94305158,-47.911561,205977.82,7238205.76,NaN,NaN,NaN,16.0,can_250821_S6_B..csv,2025-08-21 18:00:46.200000+00:00,NaN,NaN,233.0,15877.456800,-6752.508300,-15522.045600,23208.346400
79344,21-Aug-2025,18:00:46.700,22584.775,89,0.0,0,215,133,NaN,NaN,NaN,NaN,29,21-Aug-2025 18:00:47.050,Line 38,-24.94325616,-47.911861,205948.03,7238182.44,NaN,-24.94304691,-47.911555,205978.46,7238206.30,NaN,NaN,NaN,16.5,can_250821_S6_B..csv,2025-08-21 18:00:46.700000+00:00,NaN,NaN,233.0,15877.447133,-6752.506550,-15522.047933,23208.340733


In [23]:
df_all_complete.shape

(79346, 37)

In [24]:
df_all_complete.isna().sum()

,0
Reading_Date,0
Reading_Time,0
Magnetic_Field,0
Signal_Quality,0
Depth,0
Leak,0
Measurement_Time,0
Signal_Strength,0
Gradient_Condition,78303
Weak_Signal,79346


In [25]:
df_all_complete.dropna(subset=['Mag_Easting', 'Mag_Northing'], inplace=True)

In [26]:
df_all_complete.isna().sum()

,0
Reading_Date,0
Reading_Time,0
Magnetic_Field,0
Signal_Quality,0
Depth,0
Leak,0
Measurement_Time,0
Signal_Strength,0
Gradient_Condition,77376
Weak_Signal,78418


In [27]:
os.makedirs(f'{data_dir}/merged_data', exist_ok=True)

In [28]:
df_all_complete.to_csv(f'{data_dir}/merged_data/can_mag_raw.csv', index=False)

## Plotting time-series profiles

Calculating profile distances first:

In [29]:
import numpy as np
import pandas as pd

def add_profile_distance(
    df: pd.DataFrame,
    x_col: str = "Mag_Easting",
    y_col: str = "Mag_Northing",
    distance_col: str = "Profile_Dist_m",
    method: str = "planar",
    # For lat/lon mode only:
    lat_col: str = "Mag_Latitude",
    lon_col: str = "Mag_Longitude",
    include_vertical: bool = False,
    z_col: str = "Depth",   # positive down; only used if include_vertical=True
) -> pd.DataFrame:
    """
    Add along-track cumulative distance (meters) from the first valid sample.

    By default, uses planar distances in meters between successive samples:
    Δs_i = hypot( x_i - x_{i-1}, y_i - y_{i-1} )
    The cumulative profile distance S_i = Σ Δs_k starts at 0 for the first valid row.

    If `method="geodesic"`, uses a fast haversine approximation on latitude/longitude
    (degrees) with Earth radius R = 6371000 m. Optionally adds vertical increments.

    Parameters
    ----------
    df : pandas.DataFrame
        Input survey table in acquisition order (no sorting is performed).
    x_col, y_col : str, default ("Mag_Easting","Mag_Northing")
        Column names of planar coordinates in meters (e.g., UTM).
    distance_col : str, default "Profile_Dist_m"
        Name of the output cumulative distance column (meters).
    method : {"planar","geodesic"}, default "planar"
        - "planar": uses (x,y) in meters directly (recommended for UTM).
        - "geodesic": uses (lat,lon) in degrees with a haversine sphere model.
    lat_col, lon_col : str
        Column names for latitude/longitude in degrees (only for method="geodesic").
    include_vertical : bool, default False
        If True, add vertical increments into the step length:
        Δs_i = sqrt(Δx^2 + Δy^2 + Δz^2). For "geodesic", Δx^2+Δy^2 is the surface distance^2.
    z_col : str, default "Depth"
        Vertical column. If `include_vertical=True`, we use Δz = z_i - z_{i-1}.
        (No sign convention is assumed; only differences matter.)

    Returns
    -------
    pandas.DataFrame
        Copy of `df` with a new column `distance_col` in meters. Rows with missing
        coordinates produce NaN at that position; the cumulative value continues
        once valid coordinates resume.

    Notes
    -----
    - The function preserves row order and does not sort by time.
    - Any non-finite (NaN/Inf) coordinates cause a NaN at that row; the running
      cumulative distance is not reset (it picks up from the last finite value).
    - For long marine lines in UTM, "planar" is appropriate and fastest.
    """
    out = df.copy()

    if method not in ("planar", "geodesic"):
        raise ValueError("method must be 'planar' or 'geodesic'")

    if method == "planar":
        x = pd.to_numeric(out[x_col], errors="coerce").to_numpy()
        y = pd.to_numeric(out[y_col], errors="coerce").to_numpy()
        # pairwise diffs with NaN-safe handling
        dx = np.empty_like(x, dtype=float); dx[:] = np.nan
        dy = np.empty_like(y, dtype=float); dy[:] = np.nan

        # valid consecutive pairs
        m = np.isfinite(x) & np.isfinite(y)
        # shifted validity for previous sample
        m_prev = m & np.roll(m, 1)
        # compute diffs where both current and previous are valid
        dx[m_prev] = x[m_prev] - np.roll(x, 1)[m_prev]
        dy[m_prev] = y[m_prev] - np.roll(y, 1)[m_prev]

        ds = np.hypot(dx, dy)

    else:  # geodesic (haversine)
        lat = np.deg2rad(pd.to_numeric(out[lat_col], errors="coerce").to_numpy())
        lon = np.deg2rad(pd.to_numeric(out[lon_col], errors="coerce").to_numpy())
        R = 6371000.0  # meters

        dlat = np.empty_like(lat, dtype=float); dlat[:] = np.nan
        dlon = np.empty_like(lon, dtype=float); dlon[:] = np.nan

        m = np.isfinite(lat) & np.isfinite(lon)
        m_prev = m & np.roll(m, 1)

        dlat[m_prev] = lat[m_prev] - np.roll(lat, 1)[m_prev]
        dlon[m_prev] = lon[m_prev] - np.roll(lon, 1)[m_prev]

        a = np.empty_like(lat, dtype=float); a[:] = np.nan
        # haversine formula on valid pairs
        tmp = m_prev
        a[tmp] = (
            np.sin(dlat[tmp] / 2.0) ** 2
            + np.cos(lat[tmp]) * np.cos(np.roll(lat, 1)[tmp]) * np.sin(dlon[tmp] / 2.0) ** 2
        )
        c = np.empty_like(lat, dtype=float); c[:] = np.nan
        c[tmp] = 2.0 * np.arctan2(np.sqrt(a[tmp]), np.sqrt(1.0 - a[tmp]))
        ds = R * c  # surface step (meters)

    # vertical component (optional)
    if include_vertical:
        z = pd.to_numeric(out[z_col], errors="coerce").to_numpy()
        dz = np.empty_like(z, dtype=float); dz[:] = np.nan
        mz = np.isfinite(z)
        mz_prev = mz & np.roll(mz, 1)
        dz[mz_prev] = z[mz_prev] - np.roll(z, 1)[mz_prev]
        # combine (planar or geodesic surface) with vertical
        # Use where both ds and dz are finite; otherwise keep ds as-is
        both = np.isfinite(ds) & np.isfinite(dz)
        ds[both] = np.hypot(ds[both], dz[both])

    # cumulative sum (NaN-safe): treat NaN increments as 0 for accumulation,
    # but keep the output at rows with NaN coords as NaN
    inc = np.where(np.isfinite(ds), ds, 0.0)
    S = np.cumsum(inc)

    # rows with invalid coords should be NaN in the output
    valid_row = np.isfinite(ds) | (S == 0)  # first row could be 0 even if ds is NaN
    out[distance_col] = S.astype(float)
    out.loc[~valid_row, distance_col] = np.nan

    # Ensure the very first finite row starts at 0.0
    first_valid = np.argmax(np.isfinite(out[distance_col].to_numpy()))
    if np.isfinite(out.loc[first_valid, distance_col]):
        out.loc[first_valid, distance_col] = 0.0

    return out

In [30]:
df_all_complete = add_profile_distance(
    df_all_complete,
    x_col="Mag_Easting",
    y_col="Mag_Northing",
    distance_col="Profile_Dist_m",
    method="planar",         # UTM meters
    include_vertical=False   # set True to include depth differences
)

In [31]:
df_all_complete

,Reading_Date,Reading_Time,Magnetic_Field,Signal_Quality,Depth,Leak,Measurement_Time,Signal_Strength,Gradient_Condition,Weak_Signal,Poor_Reading,Instrument_Mistuned,Reading_ID,System_Time,Line_Name,Mag_Latitude,Mag_Longitude,Mag_Easting,Mag_Northing,Mag_Zone,GPS_Latitude,GPS_Longitude,GPS_Easting,GPS_Northing,GPS_Zone,Marker,Marker_Time,timestamp_s,source_file,__survey_datetime_utc,DATE_vss,TIME_vss,DOY_vss,VSSX_vss,VSSY_vss,VSSZ_vss,VSSF_vss,Profile_Dist_m
0,20-Aug-2025,17:36:36.700,22759.127,79,0.0,0,215,119,NaN,NaN,NaN,NaN,2,20-Aug-2025 17:36:38.207,Line 01,-25.04215631,-48.017993,800898.50,7227067.90,NaN,-25.04222737,-48.018367,800860.58,7227060.86,NaN,NaN,NaN,0.0,CAN_250820_S6_C.csv,2025-08-20 17:36:36.700000+00:00,NaN,NaN,232.0,15878.953200,-6747.600383,-15509.501750,23199.548250,0.000000e+00
1,20-Aug-2025,17:36:37.200,22757.742,79,0.0,0,215,122,NaN,NaN,NaN,NaN,3,20-Aug-2025 17:36:38.660,Line 01,-25.04215727,-48.017996,800898.20,7227067.80,NaN,-25.04222760,-48.018370,800860.27,7227060.84,NaN,NaN,NaN,0.5,CAN_250820_S6_C.csv,2025-08-20 17:36:37.200000+00:00,NaN,NaN,232.0,15878.951200,-6747.597800,-15509.503000,23199.547000,5.950546e+05
2,20-Aug-2025,17:36:37.700,22756.970,89,0.0,0,215,129,NaN,NaN,NaN,NaN,4,20-Aug-2025 17:36:39.157,Line 01,-25.04215968,-48.018003,800897.46,7227067.55,NaN,-25.04222816,-48.018378,800859.48,7227060.79,NaN,NaN,NaN,1.0,CAN_250820_S6_C.csv,2025-08-20 17:36:37.700000+00:00,NaN,NaN,232.0,15878.949200,-6747.595217,-15509.504250,23199.545750,5.950554e+05
3,20-Aug-2025,17:36:38.700,22753.489,89,0.0,0,215,133,NaN,NaN,NaN,NaN,5,20-Aug-2025 17:36:40.177,Line 01,-25.04216432,-48.018018,800895.97,7227067.07,NaN,-25.04222907,-48.018393,800857.90,7227060.73,NaN,NaN,NaN,2.0,CAN_250820_S6_C.csv,2025-08-20 17:36:38.700000+00:00,NaN,NaN,232.0,15878.945200,-6747.590050,-15509.506750,23199.543250,5.950569e+05
4,20-Aug-2025,17:36:39.200,22750.916,89,0.0,0,215,134,NaN,NaN,NaN,NaN,6,20-Aug-2025 17:36:40.687,Line 01,-25.04216656,-48.018025,800895.22,7227066.83,NaN,-25.04222942,-48.018401,800857.11,7227060.70,NaN,NaN,NaN,2.5,CAN_250820_S6_C.csv,2025-08-20 17:36:39.200000+00:00,NaN,NaN,232.0,15878.943200,-6747.587467,-15509.508000,23199.542000,5.950577e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79340,21-Aug-2025,18:00:44.700,22585.304,89,0.0,0,215,132,NaN,NaN,NaN,NaN,25,21-Aug-2025 18:00:45.010,Line 38,-24.94327238,-47.911890,205945.17,7238180.58,NaN,-24.94306675,-47.911581,205975.88,7238204.04,NaN,NaN,NaN,14.5,can_250821_S6_B..csv,2025-08-21 18:00:44.700000+00:00,NaN,NaN,233.0,15877.485800,-6752.513550,-15522.038600,23208.363400,9.790242e+06
79341,21-Aug-2025,18:00:45.200,22585.829,89,0.0,0,215,129,NaN,NaN,NaN,NaN,26,21-Aug-2025 18:00:45.510,Line 38,-24.94326828,-47.911882,205945.90,7238181.05,NaN,-24.94306156,-47.911574,205976.53,7238204.63,NaN,NaN,NaN,15.0,can_250821_S6_B..csv,2025-08-21 18:00:45.200000+00:00,NaN,NaN,233.0,15877.476133,-6752.511800,-15522.040933,23208.357733,9.790243e+06
79342,21-Aug-2025,18:00:45.700,22585.503,89,0.0,0,215,133,NaN,NaN,NaN,NaN,27,21-Aug-2025 18:00:46.023,Line 38,-24.94326420,-47.911875,205946.62,7238181.52,NaN,-24.94305649,-47.911568,205977.17,7238205.21,NaN,NaN,NaN,15.5,can_250821_S6_B..csv,2025-08-21 18:00:45.700000+00:00,NaN,NaN,233.0,15877.466467,-6752.510050,-15522.043267,23208.352067,9.790243e+06
79343,21-Aug-2025,18:00:46.200,22585.352,89,0.0,0,215,133,NaN,NaN,NaN,NaN,28,21-Aug-2025 18:00:46.543,Line 38,-24.94326015,-47.911868,205947.33,7238181.99,NaN,-24.94305158,-47.911561,205977.82,7238205.76,NaN,NaN,NaN,16.0,can_250821_S6_B..csv,2025-08-21 18:00:46.200000+00:00,NaN,NaN,233.0,15877.456800,-6752.508300,-15522.045600,23208.346400,9.790244e+06


In [32]:
df_all_complete.isna().sum()

,0
Reading_Date,0
Reading_Time,0
Magnetic_Field,0
Signal_Quality,0
Depth,0
Leak,0
Measurement_Time,0
Signal_Strength,0
Gradient_Condition,77376
Weak_Signal,78418


In [33]:
# # --- prerequisites (safe no-op outside Colab)
# def _enable_widgets_for_colab():
#     """
#     Enable ipywidgets support in Google Colab.

#     This helper ensures that interactive widgets (from `ipywidgets`)
#     render correctly when running inside Google Colab. It uses
#     the Colab `output.enable_custom_widget_manager()` method.
#     Safe to call in any environment—does nothing if not in Colab.
#     """
#     try:
#         from google.colab import output  # noqa: F401
#         output.enable_custom_widget_manager()
#     except Exception:
#         pass
# _enable_widgets_for_colab()

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from ipywidgets import widgets, HBox, VBox
# from IPython.display import display


# # ---------- helpers ----------
# def _ensure_datetime(df, date_col="Reading_Date", time_col="Reading_Time"):
#     """
#     Build a UTC-aware datetime Series for the survey data.

#     Parameters
#     ----------
#     df : pandas.DataFrame
#         DataFrame containing survey timestamps.
#     date_col : str, default="Reading_Date"
#         Column name containing the date (e.g., '21-Aug-2025').
#     time_col : str, default="Reading_Time"
#         Column name containing the time (e.g., '18:02:28.200').

#     Returns
#     -------
#     pandas.Series
#         Series of datetime64[ns, UTC] objects combining date and time.
#         If the DataFrame already includes a '__survey_datetime_utc' column,
#         that column is parsed instead.
#     """
#     if "__survey_datetime_utc" in df.columns:
#         dt = pd.to_datetime(df["__survey_datetime_utc"], errors="coerce", utc=True)
#     else:
#         dt = pd.to_datetime(
#             df[date_col].astype(str) + " " + df[time_col].astype(str),
#             format="%d-%b-%Y %H:%M:%S.%f",
#             errors="coerce",
#             utc=True,
#         )
#     return dt


# def _to_num(s):
#     """
#     Safely convert a pandas Series to numeric.

#     Converts string or mixed-type data into float64,
#     coercing invalid entries to NaN.

#     Parameters
#     ----------
#     s : pandas.Series
#         Input data series.

#     Returns
#     -------
#     pandas.Series
#         Numeric version of the input, with non-numeric
#         values converted to NaN.
#     """
#     return pd.to_numeric(s, errors="coerce")


# def _ordered_unique(seq):
#     """
#     Return unique elements from a sequence while preserving order.

#     Parameters
#     ----------
#     seq : iterable
#         Input sequence (list, Series, etc.).

#     Returns
#     -------
#     list
#         List of unique elements in the order of first appearance.
#     """
#     seen, out = set(), []
#     for x in seq:
#         x = "" if x is None else str(x)
#         if x not in seen:
#             seen.add(x)
#             out.append(x)
#     return out


# def _guess_geomag_F_col(df):
#     """
#     Attempt to automatically identify the geomagnetic 'F' column.

#     The function searches for likely total field columns in an
#     observatory dataset (merged with the survey), such as 'VSSF_vss'
#     or 'F_vss'.

#     Parameters
#     ----------
#     df : pandas.DataFrame
#         DataFrame containing observatory data columns.

#     Returns
#     -------
#     str or None
#         The guessed geomagnetic column name, or None if none found.
#     """
#     for name in ("VSSF_vss", "F_vss"):
#         if name in df.columns:
#             return name
#     for c in df.columns:
#         lc = c.lower()
#         if lc.endswith("f_vss") or lc.endswith("vssf_vss"):
#             return c
#     for c in df.columns:
#         if c.endswith("_vss"):
#             return c
#     return None


# # ---------- single static overview (optional) ----------
# def plot_lines_overview(df, line_col="line", lat_col="Mag_Latitude", lon_col="Mag_Longitude", s=4):
#     """
#     Plot a static map overview of all survey lines.

#     Parameters
#     ----------
#     df : pandas.DataFrame
#         DataFrame containing survey points and coordinates.
#     line_col : str, default="line"
#         Column name defining the survey line or traverse ID.
#     lat_col : str, default="Mag_Latitude"
#         Column with geographic latitude values.
#     lon_col : str, default="Mag_Longitude"
#         Column with geographic longitude values.
#     s : int, default=4
#         Marker size for scatter plot points.

#     Notes
#     -----
#     - Automatically converts coordinate columns to numeric.
#     - Draws each line in a different color.
#     - Displays legend only if 12 or fewer unique lines exist.
#     """
#     d = df.copy()
#     d[lon_col] = _to_num(d[lon_col])
#     d[lat_col] = _to_num(d[lat_col])
#     d = d.dropna(subset=[lon_col, lat_col])
#     d["_line_"] = d[line_col].astype(str).fillna("")

#     fig, ax = plt.subplots(figsize=(7, 6), dpi=110)
#     for ln, dsub in d.groupby("_line_"):
#         ax.scatter(dsub[lon_col].to_numpy(), dsub[lat_col].to_numpy(), s=s, alpha=0.7, label=str(ln))
#     ax.set_xlabel(lon_col)
#     ax.set_ylabel(lat_col)
#     ax.set_title("Survey lines — overview")
#     ax.grid(True, ls=":")
#     if d["_line_"].nunique() <= 12:
#         ax.legend(title="Line", fontsize=8)
#     plt.show()


# # ---------- utility to format date label for a line ----------
# def _line_dates_label(d_ts):
#     """
#     Return a compact UTC date label for a given survey line.

#     Parameters
#     ----------
#     d_ts : pandas.DataFrame
#         Subset of the survey DataFrame corresponding to one line,
#         containing the '__dt' datetime column.

#     Returns
#     -------
#     str
#         A short date label string (e.g. '2025-08-21 … 2025-08-22').
#         Returns 'no date' if input DataFrame is empty.
#     """
#     if d_ts.empty:
#         return "no date"
#     dates = d_ts["__dt"].dt.tz_convert("UTC").dt.date.unique()
#     dates = np.sort(dates.astype("datetime64[D]")).astype(object)
#     if len(dates) == 1:
#         return str(dates[0])
#     return f"{dates[0]} … {dates[-1]}"


# # ---------- interactive browser (map + time series viewer) ----------
# import matplotlib.ticker as mticker

# def make_line_browser(
#     df,
#     line_col="line",
#     date_col="Reading_Date",
#     time_col="Reading_Time",
#     lat_col="Mag_Latitude",
#     lon_col="Mag_Longitude",
#     survey_col="Magnetic_Field",
#     geomag_col=None,
#     value_cols=None,
# ):
#     """
#     Create an interactive browser to explore survey lines and plot two selectable channels.

#     Panels:
#       - Map of survey tracks with selected line highlighted.
#       - Time series with two user-selectable channels (Channel A and Channel B).
#         Channel B can be disabled by selecting "(none)".

#     Parameters
#     ----------
#     df : pandas.DataFrame
#         Survey DataFrame with timestamps, coordinates, line IDs, and signals.
#     line_col : str, default="line"
#         Survey line identifier column.
#     date_col, time_col : str
#         Date/time strings to build UTC timestamps if __survey_datetime_utc is absent.
#     lat_col, lon_col : str
#         Coordinate columns used in the map.
#     survey_col : str
#         Default primary series shown in Channel A.
#     geomag_col : str or None
#         Default secondary series shown in Channel B. If None, function will try to guess it.
#     value_cols : list[str] or None
#         If provided, restricts the Channel dropdown options to this list.
#         If None, numeric-like columns are auto-detected (excluding coordinates and obvious text columns).
#     """
#     d = df.copy()
#     d["__dt"] = _ensure_datetime(d, date_col=date_col, time_col=time_col)
#     d[lat_col] = _to_num(d[lat_col]); d[lon_col] = _to_num(d[lon_col])
#     d[survey_col] = _to_num(d[survey_col])

#     # Default/guess geomag col
#     if geomag_col is None:
#         geomag_col = _guess_geomag_F_col(d)
#     if geomag_col in d.columns:
#         d[geomag_col] = _to_num(d[geomag_col])
#     else:
#         geomag_col = None

#     # Clean for map & labels
#     d_map = d.dropna(subset=[lat_col, lon_col]).copy()
#     d["_line_"] = d[line_col].astype(str).fillna("")
#     d_map["_line_"] = d_map[line_col].astype(str).fillna("")

#     # Stable list of lines
#     line_list = _ordered_unique(d["_line_"].tolist())
#     if not line_list:
#         raise ValueError(f"No lines found in '{line_col}'")

#     # Build list of numeric-like columns for plotting choices
#     if value_cols is None:
#         drop_like = {line_col, date_col, time_col, "__dt", "__survey_datetime_utc",
#                      lat_col, lon_col, "Marker", "Marker_Time", "System_Time"}
#         candidates = []
#         for c in d.columns:
#             if c in drop_like:
#                 continue
#             s = pd.to_numeric(d[c].head(300), errors="coerce")
#             if s.notna().mean() > 0.6:
#                 candidates.append(c)
#         for defc in (survey_col, geomag_col):
#             if defc and defc in d.columns and defc not in candidates:
#                 candidates.append(defc)
#         value_cols = sorted(set(candidates))
#     value_cols_with_none = ["(none)"] + value_cols

#     # --- widgets ---
#     slider = widgets.SelectionSlider(
#         options=line_list, value=line_list[0],
#         description="Line:", continuous_update=False,
#         layout=widgets.Layout(width="45%")
#     )

#     chA_dd = widgets.Dropdown(
#         options=value_cols, value=survey_col if survey_col in value_cols else (value_cols[0] if value_cols else None),
#         description="Channel A:", layout=widgets.Layout(width="45%")
#     )
#     chB_initial = geomag_col if (geomag_col and geomag_col in value_cols) else "(none)"
#     chB_dd = widgets.Dropdown(
#         options=value_cols_with_none, value=chB_initial,
#         description="Channel B:", layout=widgets.Layout(width="45%")
#     )

#     first_ts = d[d["_line_"] == line_list[0]].dropna(subset=["__dt"])
#     info = widgets.HTML(f"<b>Selected:</b> {line_list[0]} &nbsp; | &nbsp; <b>Date:</b> {_line_dates_label(first_ts)}")
#     out = widgets.Output()

#     def redraw(selected_line, colA, colB):
#         """Redraw map + time series with the selected line and channels."""
#         with out:
#             out.clear_output(wait=True)
#             fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(11, 6.5), dpi=130, constrained_layout=True)

#             # Data for this line
#             d_ts = d[d["_line_"] == selected_line].dropna(subset=["__dt"]).sort_values("__dt")
#             date_label = _line_dates_label(d_ts)

#             # --- Map ---
#             ax0 = axes[0]
#             ax0.scatter(d_map[lon_col], d_map[lat_col], s=2, label="All")
#             d_line_map = d_map[d_map["_line_"] == selected_line]
#             if not d_line_map.empty:
#                 ax0.scatter(d_line_map[lon_col], d_line_map[lat_col], s=6, label=selected_line)
#             ax0.set_xlabel(lon_col); ax0.set_ylabel(lat_col)
#             ax0.set_title(f"Track — Line: {selected_line}  |  Date: {date_label}")
#             ax0.grid(True, ls=":"); ax0.legend(loc="best", fontsize=8)

#             # --- Time series ---
#             ax1 = axes[1]
#             if not d_ts.empty:
#                 if colA and colA in d_ts.columns:
#                     ax1.plot(d_ts["__dt"], pd.to_numeric(d_ts[colA], errors="coerce"), lw=1.2, label=colA)
#                 if colB and (colB != "(none)") and (colB in d_ts.columns):
#                     ax1.plot(d_ts["__dt"], pd.to_numeric(d_ts[colB], errors="coerce"), lw=1.0, alpha=0.9, label=colB)

#             ax1.set_xlabel("Time (UTC)")
#             ax1.set_ylabel("Value")
#             ax1.set_title(f"Time series — Line: {selected_line}  |  Date: {date_label}")
#             ax1.grid(True, ls=":")
#             ax1.legend(loc="best", fontsize=8)

#             # ---- avoid scientific notation ----
#             ax1.yaxis.set_major_formatter(mticker.ScalarFormatter(useMathText=False))
#             ax1.ticklabel_format(style='plain', axis='y')
#             ax1.yaxis.get_major_locator().set_params(integer=True)

#             plt.show()

#     # Initial draw
#     redraw(slider.value, chA_dd.value, chB_dd.value)

#     # Callbacks
#     def on_line_change(change):
#         if change["name"] == "value" and change["type"] == "change":
#             sel = change["new"]
#             d_ts_sel = d[d["_line_"] == sel].dropna(subset=["__dt"])
#             info.value = f"<b>Selected:</b> {sel} &nbsp; | &nbsp; <b>Date:</b> {_line_dates_label(d_ts_sel)}"
#             redraw(sel, chA_dd.value, chB_dd.value)

#     def on_chA_change(change):
#         if change["name"] == "value":
#             redraw(slider.value, change["new"], chB_dd.value)

#     def on_chB_change(change):
#         if change["name"] == "value":
#             redraw(slider.value, chA_dd.value, change["new"])

#     slider.observe(on_line_change, names="value")
#     chA_dd.observe(on_chA_change, names="value")
#     chB_dd.observe(on_chB_change, names="value")

#     # Layout
#     controls_top = HBox([slider, info])
#     controls_bottom = HBox([chA_dd, chB_dd])
#     display(VBox([controls_top, controls_bottom, out]))


In [34]:
# --- prerequisites (safe no-op outside Colab)
def _enable_widgets_for_colab():
    """
    Enable ipywidgets support in Google Colab.

    This helper ensures that interactive widgets (from `ipywidgets`)
    render correctly when running inside Google Colab. It uses
    the Colab `output.enable_custom_widget_manager()` method.
    Safe to call in any environment—does nothing if not in Colab.
    """
    try:
        from google.colab import output  # noqa: F401
        output.enable_custom_widget_manager()
    except Exception:
        pass
_enable_widgets_for_colab()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, HBox, VBox
from IPython.display import display
import matplotlib.ticker as mticker


# ---------- helpers ----------
def _ensure_datetime(df, date_col="Reading_Date", time_col="Reading_Time"):
    """
    Build a UTC-aware datetime Series for the survey data.

    If '__survey_datetime_utc' exists, that is used; otherwise parses
    date_col + time_col (format '%d-%b-%Y %H:%M:%S.%f').
    """
    if "__survey_datetime_utc" in df.columns:
        dt = pd.to_datetime(df["__survey_datetime_utc"], errors="coerce", utc=True)
    else:
        dt = pd.to_datetime(
            df[date_col].astype(str) + " " + df[time_col].astype(str),
            format="%d-%b-%Y %H:%M:%S.%f",
            errors="coerce",
            utc=True,
        )
    return dt


def _to_num(s):
    """Numeric coercion with errors='coerce'."""
    return pd.to_numeric(s, errors="coerce")


def _ordered_unique(seq):
    """Unique in first-appearance order."""
    seen, out = set(), []
    for x in seq:
        x = "" if x is None else str(x)
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out


def _guess_geomag_F_col(df):
    """
    Guess a geomag total-field column (observatory) if present.
    """
    for name in ("VSSF_vss", "F_vss"):
        if name in df.columns:
            return name
    for c in df.columns:
        lc = c.lower()
        if lc.endswith("f_vss") or lc.endswith("vssf_vss"):
            return c
    for c in df.columns:
        if c.endswith("_vss"):
            return c
    return None


def _line_dates_label(d_ts):
    """Compact UTC date label for a line subset that contains '__dt'."""
    if d_ts.empty:
        return "no date"
    dates = d_ts["__dt"].dt.tz_convert("UTC").dt.date.unique()
    dates = np.sort(dates.astype("datetime64[D]")).astype(object)
    if len(dates) == 1:
        return str(dates[0])
    return f"{dates[0]} … {dates[-1]}"


def _profile_distance_from_xy(dsub, x_col="Mag_Easting", y_col="Mag_Northing"):
    """
    Compute along-track cumulative distance (meters) for a single line subset.

    Returns a numpy array S (same length as dsub, in current row order),
    with the first finite point set to 0, NaN where coords are invalid.
    """
    x = _to_num(dsub[x_col]).to_numpy()
    y = _to_num(dsub[y_col]).to_numpy()

    dx = np.full_like(x, np.nan, dtype=float)
    dy = np.full_like(y, np.nan, dtype=float)

    m = np.isfinite(x) & np.isfinite(y)
    if not m.any():
        return np.full(len(dsub), np.nan, dtype=float)

    m_prev = m & np.roll(m, 1)
    dx[m_prev] = x[m_prev] - np.roll(x, 1)[m_prev]
    dy[m_prev] = y[m_prev] - np.roll(y, 1)[m_prev]

    ds = np.hypot(dx, dy)
    inc = np.where(np.isfinite(ds), ds, 0.0)
    S = np.cumsum(inc)
    # keep NaN where coords invalid
    S[~m] = np.nan
    # force first finite to 0
    first_valid = np.argmax(m)
    if m[first_valid]:
        S[first_valid] = 0.0
    return S


# ---------- optional overview map ----------
def plot_lines_overview(df, line_col="line", lat_col="Mag_Latitude", lon_col="Mag_Longitude", s=4):
    """
    Static map overview of lines (scatter by lon/lat).
    """
    d = df.copy()
    d[lon_col] = _to_num(d[lon_col])
    d[lat_col] = _to_num(d[lat_col])
    d = d.dropna(subset=[lon_col, lat_col])
    d["_line_"] = d[line_col].astype(str).fillna("")

    fig, ax = plt.subplots(figsize=(7, 6), dpi=110)
    for ln, dsub in d.groupby("_line_"):
        ax.scatter(dsub[lon_col].to_numpy(), dsub[lat_col].to_numpy(), s=s, alpha=0.7, label=str(ln))
    ax.set_xlabel(lon_col)
    ax.set_ylabel(lat_col)
    ax.set_title("Survey lines — overview")
    ax.grid(True, ls=":")
    if d["_line_"].nunique() <= 12:
        ax.legend(title="Line", fontsize=8)
    plt.show()


# ---------- interactive browser (map + time/distance series viewer) ----------
def make_line_browser(
    df,
    line_col="line",
    date_col="Reading_Date",
    time_col="Reading_Time",
    lat_col="Mag_Latitude",
    lon_col="Mag_Longitude",
    survey_col="Magnetic_Field",
    geomag_col=None,
    value_cols=None,
    # NEW: use your precomputed distance column
    distance_col="Profile_Dist_m",
    sort_distance=True,   # sort by distance when plotting distance profiles
):
    """
    Interactive browser to explore survey lines and plot vs Time (UTC) or precomputed Distance (m).

    Parameters
    ----------
    df : DataFrame
        Your survey data (already includes the precomputed distance column).
    line_col : str
        Column with line identifiers.
    date_col, time_col : str
        Columns for date and time; combined to a UTC-aware datetime if '__survey_datetime_utc' not present.
    lat_col, lon_col : str
        Latitude/Longitude columns for map view.
    survey_col : str
        Default primary channel to plot on the series panel.
    geomag_col : str or None
        Optional geomagnetic observatory total-field column; auto-guessed if None.
    value_cols : list[str] or None
        If None, inferred from numeric-like columns with sufficient valid values.
    distance_col : str, default "Profile_Dist_m"
        Column with along-track distance in meters (precomputed by you).
    sort_distance : bool, default True
        If True, sort the selected line by distance before plotting in Distance mode.
    """
    d = df.copy()
    d["__dt"] = _ensure_datetime(d, date_col=date_col, time_col=time_col)
    d[lat_col] = _to_num(d[lat_col]); d[lon_col] = _to_num(d[lon_col])
    d[survey_col] = _to_num(d[survey_col])

    # Default/guess geomag col
    if geomag_col is None:
        geomag_col = _guess_geomag_F_col(d)
    if geomag_col in d.columns:
        d[geomag_col] = _to_num(d[geomag_col])
    else:
        geomag_col = None

    # Clean for map & labels
    d_map = d.dropna(subset=[lat_col, lon_col]).copy()
    d["_line_"] = d[line_col].astype(str).fillna("")
    d_map["_line_"] = d_map[line_col].astype(str).fillna("")

    # Stable list of lines
    line_list = _ordered_unique(d["_line_"].tolist())
    if not line_list:
        raise ValueError(f"No lines found in '{line_col}'")

    # Build list of numeric-like columns for plotting choices
    if value_cols is None:
        drop_like = {
            line_col, date_col, time_col, "__dt", "__survey_datetime_utc",
            lat_col, lon_col, "Marker", "Marker_Time", "System_Time"
        }
        candidates = []
        for c in d.columns:
            if c in drop_like:
                continue
            s = pd.to_numeric(d[c].head(300), errors="coerce")
            if s.notna().mean() > 0.6:
                candidates.append(c)
        for defc in (survey_col, geomag_col):
            if defc and defc in d.columns and defc not in candidates:
                candidates.append(defc)
        value_cols = sorted(set(candidates))
    value_cols_with_none = ["(none)"] + value_cols

    # --- widgets ---
    # DROPDOWN (replaces SelectionSlider to avoid float index bug in Colab)
    slider = widgets.Dropdown(
        options=line_list,
        value=line_list[0],
        description="Line:",
        layout=widgets.Layout(width="40%")
    )

    axis_mode = widgets.ToggleButtons(
        options=[("Time (UTC)", "time"), ("Distance (m)", "dist")],
        value="time",
        description="X-axis:",
        layout=widgets.Layout(width="40%")
    )

    chA_dd = widgets.Dropdown(
        options=value_cols,
        value=survey_col if survey_col in value_cols else (value_cols[0] if value_cols else None),
        description="Channel A:", layout=widgets.Layout(width="45%")
    )
    chB_initial = geomag_col if (geomag_col and geomag_col in value_cols) else "(none)"
    chB_dd = widgets.Dropdown(
        options=value_cols_with_none, value=chB_initial,
        description="Channel B:", layout=widgets.Layout(width="45%")
    )

    first_ts = d[d["_line_"] == line_list[0]].dropna(subset=["__dt"])
    info = widgets.HTML(f"<b>Selected:</b> {line_list[0]} &nbsp; | &nbsp; <b>Date:</b> {_line_dates_label(first_ts)}")
    out = widgets.Output()

    def _prep_series(d_ts, mode):
        """
        Return (x, xlabel, d_plot) for the chosen x-axis mode.
        - 'time': x = '__dt'
        - 'dist': x = numeric 'distance_col' (precomputed), optionally sorted.
        """
        if mode == "time":
            d_plot = d_ts.copy()
            x = d_plot["__dt"]
            xlabel = "Time (UTC)"
            return x, xlabel, d_plot

        # distance mode (use your column)
        if distance_col not in d_ts.columns:
            # fallback to time if missing
            d_plot = d_ts.copy()
            x = d_plot["__dt"]
            return x, "Time (UTC)", d_plot

        d_plot = d_ts.copy()
        d_plot["__dist__"] = _to_num(d_plot[distance_col])
        # keep finite distances only for plotting
        d_plot = d_plot[np.isfinite(d_plot["__dist__"])]
        if sort_distance:
            d_plot = d_plot.sort_values("__dist__")
        x = d_plot["__dist__"].to_numpy()
        xlabel = f"Distance along line (m) [{distance_col}]"
        return x, xlabel, d_plot

    def redraw(selected_line, colA, colB, mode):
        with out:
            out.clear_output(wait=True)
            fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(11, 6.7), dpi=130, constrained_layout=True)

            # Data for this line (time-sorted for stable behavior)
            d_ts = d[d["_line_"] == selected_line].dropna(subset=["__dt"]).sort_values("__dt")
            date_label = _line_dates_label(d_ts)

            # --- Map ---
            ax0 = axes[0]
            ax0.scatter(d_map[lon_col], d_map[lat_col], s=2, label="All")
            d_line_map = d_map[d_map["_line_"] == selected_line]
            if not d_line_map.empty:
                ax0.scatter(d_line_map[lon_col], d_line_map[lat_col], s=6, label=selected_line)
            ax0.set_xlabel(lon_col); ax0.set_ylabel(lat_col)
            ax0.set_title(f"Track — Line: {selected_line}  |  Date: {date_label}")
            ax0.grid(True, ls=":"); ax0.legend(loc="best", fontsize=8)

            # --- Series (Time or Distance) ---
            ax1 = axes[1]
            if not d_ts.empty:
                xvals, xlabel, d_plot = _prep_series(d_ts, mode)
                if colA and colA in d_plot.columns:
                    ax1.plot(xvals, _to_num(d_plot[colA]), lw=1.2, label=colA)
                if colB and (colB != "(none)") and (colB in d_plot.columns):
                    ax1.plot(xvals, _to_num(d_plot[colB]), lw=1.0, alpha=0.9, label=colB)
                ax1.set_xlabel(xlabel)
            else:
                ax1.set_xlabel("Time (UTC)")

            ax1.set_ylabel("Value")
            ax1.set_title(f"{'Distance' if mode=='dist' else 'Time'} series — Line: {selected_line}  |  Date: {date_label}")
            ax1.grid(True, ls=":")
            ax1.legend(loc="best", fontsize=8)

            # avoid scientific notation on Y; keep integer ticks if suitable
            ax1.yaxis.set_major_formatter(mticker.ScalarFormatter(useMathText=False))
            ax1.ticklabel_format(style='plain', axis='y')
            ax1.yaxis.set_major_locator(mticker.MaxNLocator(integer=True))

            plt.show()

    # Initial draw
    redraw(slider.value, chA_dd.value, chB_dd.value, axis_mode.value)

    # Callbacks
    def on_line_change(change):
        if change["name"] == "value" and change["type"] == "change":
            sel = change["new"]
            d_ts_sel = d[d["_line_"] == sel].dropna(subset=["__dt"])
            info.value = f"<b>Selected:</b> {sel} &nbsp; | &nbsp; <b>Date:</b> {_line_dates_label(d_ts_sel)}"
            redraw(sel, chA_dd.value, chB_dd.value, axis_mode.value)

    def on_chA_change(change):
        if change["name"] == "value":
            redraw(slider.value, change["new"], chB_dd.value, axis_mode.value)

    def on_chB_change(change):
        if change["name"] == "value":
            redraw(slider.value, chA_dd.value, change["new"], axis_mode.value)

    def on_axis_mode_change(change):
        if change["name"] == "value":
            redraw(slider.value, chA_dd.value, chB_dd.value, change["new"])

    slider.observe(on_line_change, names="value")
    chA_dd.observe(on_chA_change, names="value")
    chB_dd.observe(on_chB_change, names="value")
    axis_mode.observe(on_axis_mode_change, names="value")

    # Layout
    controls_top = HBox([slider, axis_mode, info])
    controls_bottom = HBox([chA_dd, chB_dd])
    display(VBox([controls_top, controls_bottom, out]))

In [35]:
make_line_browser(
    df_all_complete,
    line_col="Line_Name",
    date_col="Reading_Date", time_col="Reading_Time",
    lat_col="Mag_Latitude", lon_col="Mag_Longitude",
    survey_col="Magnetic_Field",
    geomag_col="VSSF_vss",
    distance_col="Profile_Dist_m"
    )

In [36]:
make_line_browser(df_all_complete,
                  line_col="Line_Name",
                  date_col="Reading_Date", time_col="Reading_Time",
                  lat_col="Mag_Northing", lon_col="Mag_Easting",
                  survey_col="Magnetic_Field",
                  geomag_col="VSSF_vss",
                  distance_col="Profile_Dist_m"
                  )

### Re-calculate Lat and Long from UTM coordinates

In [37]:
import geopandas as gpd

# df: your pandas DataFrame with columns: Mag_Easting, Mag_Northing
gdf_all = gpd.GeoDataFrame(
    df_all_complete,
    geometry=gpd.points_from_xy(df_all_complete["Mag_Easting"], df_all_complete["Mag_Northing"]),
    crs="EPSG:32723"  # WGS84 / UTM zone 23S
).to_crs(4326)        # WGS84 geographic

df_all_complete["Mag_Longitude"] = gdf_all.geometry.x
df_all_complete["Mag_Latitude"]  = gdf_all.geometry.y

In [38]:
make_line_browser(df_all_complete,
                  line_col="Line_Name",
                  date_col="Reading_Date", time_col="Reading_Time",
                  lat_col="Mag_Northing", lon_col="Mag_Easting",
                  survey_col="Magnetic_Field",
                  geomag_col="VSSF_vss",
                  distance_col="Profile_Dist_m"
                  )

## Data Statistics

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, VBox, HBox
from IPython.display import display
from matplotlib.ticker import ScalarFormatter

def make_histogram_browser_select(
    df,
    value_cols=None,                 # list of fields user can choose; if None, auto-detect mostly-numeric cols
    date_col="Reading_Date",
    line_col="Line_Name",
    default_bins=40
):
    """
    Interactive histogram:
      • Dropdown: Reading_Date (by day)
      • Dropdown: Line_Name (restricted to selected date)
      • Dropdown: Field to plot
      • Control: bins (int)
      • Control: Frequency vs Density
      • Overlays: mean, median, and mean±std band
    """

    d = df.copy()

    # ---- Build a pure date column (no time, no tz) ----
    if not np.issubdtype(d[date_col].dtype, np.datetime64):
        d["_date_"] = pd.to_datetime(d[date_col], errors="coerce")
    else:
        d["_date_"] = d[date_col]
    if isinstance(d["_date_"].dtype, pd.DatetimeTZDtype):
        d["_date_"] = d["_date_"].dt.tz_localize(None)
    d["_date_"] = d["_date_"].dt.normalize()
    d["_date_only_"] = d["_date_"].dt.date

    # ---- Candidate fields (auto-detect if not provided) ----
    if value_cols is None:
        numeric_guess = []
        for c in d.columns:
            if c in [date_col, line_col, "_date_", "_date_only_"]:
                continue
            s = pd.to_numeric(d[c].head(300), errors="coerce")
            if s.notna().mean() > 0.7:
                numeric_guess.append(c)
        # remove obvious non-measurement text-ish
        drop_like = {"Reading_Time", "System_Time", "Marker", "Marker_Time"}
        value_cols = [c for c in numeric_guess if c not in drop_like]
        if not value_cols:
            raise ValueError("Could not auto-detect numeric fields; pass value_cols=[...].")

    # ---- Ordered unique dates (preserve first appearance) ----
    seen, date_opts = set(), []
    for dt in d["_date_only_"]:
        if pd.isna(dt):
            continue
        if dt not in seen:
            seen.add(dt); date_opts.append(dt)
    if not date_opts:
        raise ValueError(f"No valid dates in '{date_col}'.")

    # ---- Helpers ----
    def _lines_for_date(dt):
        sub = d.loc[d["_date_only_"] == dt, line_col].astype(str)
        seen2, out = set(), []
        for x in sub:
            if x not in seen2:
                seen2.add(x); out.append(x)
        return out if out else ["(no lines)"]

    # ---- Widgets (selection boxes) ----
    date_dd   = widgets.Dropdown(options=date_opts, value=date_opts[0], description="Date:")
    line_dd   = widgets.Dropdown(options=_lines_for_date(date_dd.value),
                                 value=_lines_for_date(date_dd.value)[0],
                                 description="Line:")
    field_dd  = widgets.Dropdown(options=list(value_cols), value=value_cols[0], description="Field:")
    bins_box  = widgets.BoundedIntText(value=int(default_bins), min=5, max=200, step=1, description="Bins:")
    mode_dd   = widgets.Dropdown(options=[("Frequency", False), ("Density", True)],
                                 value=False, description="Y-axis:")

    out = widgets.Output()

    # ---- Plotting routine ----
    def redraw():
        with out:
            out.clear_output(wait=True)

            sel_date  = date_dd.value
            sel_line  = line_dd.value
            sel_field = field_dd.value
            bins      = int(bins_box.value)
            density   = bool(mode_dd.value)

            # Subset strictly to (date, line)
            mask = (d["_date_only_"] == sel_date) & (d[line_col].astype(str) == str(sel_line))
            sub = pd.to_numeric(d.loc[mask, sel_field], errors="coerce").dropna()

            fig, ax = plt.subplots(figsize=(9, 5.2), dpi=130)

            if len(sub):
                ax.hist(sub.to_numpy(), bins=bins, edgecolor='black', alpha=0.85, density=density)

                mu  = sub.mean()
                med = sub.median()
                sd  = sub.std(ddof=1)  # sample std

                # mean ± std band
                ax.axvspan(mu - sd, mu + sd, alpha=0.15, label="mean ± 1σ")

                # mean and median lines
                ax.axvline(mu,  linestyle='-',  linewidth=1.5, label=f"mean = {mu:.2f}")
                ax.axvline(med, linestyle='--', linewidth=1.5, label=f"median = {med:.2f}")

                # annotate stats in a small box
                n = len(sub)
                txt = (f"n = {n}\n"
                       f"mean = {mu:.3f}\n"
                       f"median = {med:.3f}\n"
                       f"std = {sd:.3f}")
                ax.text(0.98, 0.95, txt, ha="right", va="top", transform=ax.transAxes,
                        bbox=dict(boxstyle="round", facecolor="white", alpha=0.7), fontsize=9)

                ttl_y = "Density" if density else "Frequency"
                ax.set_title(f"{sel_field} — {sel_date} | {sel_line}")
                ax.set_ylabel(ttl_y)
            else:
                ax.text(0.5, 0.5, "No data for this (date, line).",
                        ha="center", va="center", transform=ax.transAxes)
                ax.set_title(f"{sel_field} — {sel_date} | {sel_line}")

            ax.set_xlabel(sel_field)
            ax.grid(True, linestyle=":", alpha=0.6)
            # Plain numbers on x-axis
            ax.ticklabel_format(style='plain', axis='x')
            ax.xaxis.set_major_formatter(ScalarFormatter(useOffset=False))
            ax.xaxis.get_major_formatter().set_scientific(False)

            ax.legend(loc="best", fontsize=8)
            plt.show()

    # ---- Wiring ----
    def on_date_change(change):
        if change["name"] == "value":
            new_date = change["new"]
            opts = _lines_for_date(new_date)
            line_dd.options = opts
            line_dd.value   = opts[0]
            redraw()

    for w in (date_dd, line_dd, field_dd, bins_box, mode_dd):
        w.observe(lambda ch: redraw() if ch["name"] == "value" else None, names="value")
    date_dd.observe(on_date_change, names="value")

    # ---- First draw ----
    redraw()
    controls_row1 = HBox([date_dd, line_dd])
    controls_row2 = HBox([field_dd, bins_box, mode_dd])
    display(VBox([controls_row1, controls_row2, out]))


In [40]:
make_histogram_browser_select(
    df_all_complete,
    date_col="Reading_Date",
    line_col="Line_Name",
    default_bins=40
)

## Exporting the merged data

In [41]:
df_all_complete.to_csv(f"{data_dir}/merged_data/can_mag_raw.csv", index=False)

## Assigning GCS info to the data

In [42]:
gdf_all.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [43]:
gdf_all.to_crs(32723, inplace=True)
gdf_all.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 48°W and 42°W, southern hemisphere between 80°S and equator, onshore and offshore. Brazil.
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Exporting to point shapefile

In [44]:
gdf_all.to_file(f"{data_dir}/merged_data/can_mag_raw_points.shp", driver="ESRI Shapefile", engine="pyogrio")

/tmp/ipython-input-71831052.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_all.to_file(f"{data_dir}/merged_data/can_mag_raw_points.shp", driver="ESRI Shapefile", engine="pyogrio")
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Reading_Date' to 'Reading_Da'
  ogr_write(
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Reading_Time' to 'Reading_Ti'
  ogr_write(
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Magnetic_Field' to 'Magnetic_F'
  ogr_write(
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Signal_Quality' to 'Signal_Qua'
  ogr_write(
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Measurement_Time' to 'Measure

# Visualizing the data

In [45]:
import plotly.express as px
import geopandas as gpd
import numpy as np

# Filter out rows with NaN in Mag_Latitude or Mag_Longitude
gdf_all_filtered = gdf_all.dropna(subset=['Mag_Latitude', 'Mag_Longitude']).copy()

# Convert latitude and longitude to numeric, coercing errors
gdf_all_filtered['Mag_Latitude'] = pd.to_numeric(gdf_all_filtered['Mag_Latitude'], errors='coerce')
gdf_all_filtered['Mag_Longitude'] = pd.to_numeric(gdf_all_filtered['Mag_Longitude'], errors='coerce')
gdf_all_filtered['Magnetic_Field'] = pd.to_numeric(gdf_all_filtered['Magnetic_Field'], errors='coerce')


# Filter out any rows where conversion to numeric resulted in NaN for position or magnetic field
gdf_all_filtered = gdf_all_filtered.dropna(subset=['Mag_Latitude', 'Mag_Longitude', 'Magnetic_Field']).copy()

# Calculate the extent of the data
min_lat, max_lat = gdf_all_filtered.Mag_Latitude.min(), gdf_all_filtered.Mag_Latitude.max()
min_lon, max_lon = gdf_all_filtered.Mag_Longitude.min(), gdf_all_filtered.Mag_Longitude.max()

# Add a small buffer to the extent
lat_buffer = (max_lat - min_lat) * 0.1
lon_buffer = (max_lon - min_lon) * 0.1

# Calculate the mean and standard deviation of the magnetic field data
mean_mag = gdf_all_filtered['Magnetic_Field'].mean()
std_mag = gdf_all_filtered['Magnetic_Field'].std()

# Set the color range to emphasize variations around the mean (e.g., +/- 2 standard deviations)
color_min = mean_mag - 2 * std_mag
color_max = mean_mag + 2 * std_mag

fig = px.scatter_geo(gdf_all_filtered, # Use the full filtered data
                     lat=gdf_all_filtered.Mag_Latitude,
                     lon=gdf_all_filtered.Mag_Longitude,
                     color=gdf_all_filtered.Magnetic_Field,
                     hover_name=gdf_all_filtered.Line_Name,
                     color_continuous_scale=px.colors.sequential.Viridis, # Or another suitable scale
                     range_color=[22400,23000] # Set the color axis range
                     )

# Update the layout to set the geographical extent
fig.update_layout(
    geo = dict(
        scope = 'south america', # You can adjust the scope if needed, or remove for auto-scoping based on data
        lataxis = dict(range = [min_lat - lat_buffer, max_lat + lat_buffer], dtick = 0.1), # Adjusted dtick for potentially smaller areas
        lonaxis = dict(range = [min_lon - lon_buffer, max_lon + lon_buffer], dtick = 0.1), # Adjusted dtick
        projection_type = "mercator" # Or another suitable projection
    )
)

fig.show()

Output hidden; open in https://colab.research.google.com to view.